# Workflow Specification

A SoS script can specify one or more workflows. Each workflow consists of one or more numbered steps. The numbers (should be non-negative) specify the **logical order** by which the steps are executed, but a later step might be executed before the completion of previous steps if it does not depend on the output of these steps.

This tutorial shows you how to define steps in a workflow and how to construct nested and combined workflows from single workflows. Although a Jupyter notebook, because of its interactive nature, is rarely used to execute complete workflows, we define workflows in notebook cells and execute them with options passed from cell magics such as `%set` and `%run`. Briefly speaking, `%set` sets persistent and global command line options and `%run` sets additional temporary options for the current cell. For example, the following command sets verbosity level to 2 so that SoS would display log messages of steps executed.

In [1]:
# set global logging level to INFO to display step executed. 
%set -v2

sos options is set to "-v2"


## Single default workflow

Each step of a workflow starts with a **step header** in the format of `[step_name: options]`. A single workflow can be specified without a name in a SoS script. For example, the following sections specify a workflow with four steps `5`, `10`, `20`, and `100`. As you can see, the workflow steps can be specified in any order and do not have to be consecutive (which is actually preferred because it allows easy insertion of extra steps).

In [2]:
[5]
[20]
[10]
[100]

INFO: Running default_5: 
INFO: Running default_10: 
INFO: Running default_20: 
INFO: Running default_100: 


Workflows specified in this way is the `default` workflow and are actually called `default` in SoS output. If you want to give it a meaningful name, you can specify the steps as

In [3]:
[mapping_5]
[mapping_20]
[mapping_10]
[mapping_100]

INFO: Running mapping_5: 
INFO: Running mapping_10: 
INFO: Running mapping_20: 
INFO: Running mapping_100: 


Because this SoS script defines only one workflow (`mapping`), you do not have to specify the name of workflow from SoS command

```bash
sos run myscript --input input1.fasta
```

The index of a step can be ignored if it is the only step of a workflow. 

In [4]:
[mapping]

INFO: Running mapping_0: 


## Short and long descriptions of steps

You can give each step a short description by adding a short description in parenthesis after step number.

In [5]:
[20 (mapping reads)]
[10 (initialize)]

INFO: Running default_10 (initialize): 
INFO: Running default_20 (mapping reads): 


The first comment block of each step is considered as description of the step and will be displayed when the step is executed.

In [6]:
[10 (initialize)]
# Validate input files and check available
# tools

# this step is actually empty

[20 (mapping reads)]
# Map reads using specified alignment tool

INFO: Running default_10 (initialize): Validate input files and check available tools
INFO: Running default_20 (mapping reads): Map reads using specified alignment tool


## Multiple workflows

A SoS script can define multiple workflows. For example, the following sections of SoS script defines two workflows named ``mouse`` and ``human``. 

In [7]:
%run mouse
[mouse_10]
[mouse_20]
[mouse_30]
[human_10]
[human_20]
[human_30]

INFO: Running mouse_10: 
INFO: Running mouse_20: 
INFO: Running mouse_30: 


In this case, a command line option is needed to specify workflow name. This can be done by magic `%run` in Jupyter notebook, or a positional argument from the command line, e.g.

```
    % sos run myscript mouse --input input1.fasta
```

If you would like to define a ``default`` and a named workflow, you can define them as

In [8]:
[10]
[20]
[30]
[test_10]
[test_20]
[test_30]

INFO: Running default_10: 
INFO: Running default_20: 
INFO: Running default_30: 


The `default` workflow will be executed by default using command

```bash
    % sos run myscript
```

The `test` workflow will be executed if its name is specified from the command line

```bash
    % sos run myscript test
```

## Shared workflow steps

The most common motivation of defining multiple workflows in a single SoS script is that they share certain processing steps. If this is the case, you can define sections such as

In [9]:
%run mouse
[mouse_10,human_10]
[mouse_20]
[human_20]
[mouse_30,human_30]

INFO: Running mouse_10: 
INFO: Running mouse_20: 
INFO: Running mouse_30: 


or

In [10]:
%run mouse
[*_10]
[mouse_20]
[human_20]
[*_30]

INFO: Running mouse_10: 
INFO: Running mouse_20: 
INFO: Running mouse_30: 


In [11]:
%run fly
[*_10]
[mouse_20,human_20]
[fly_20]
[*_30,fly_50]
[fly_40]


INFO: Running fly_10: 
INFO: Running fly_20: 
INFO: Running fly_30: 
INFO: Running fly_40: 
INFO: Running fly_50: 


In the last case, step defined by `[*_30,fly_40]` will be expanded to ``mouse_30``, ``human_30``, ``fly_30``, and ``fly_50`` and will be executed twice for the `fly` workflow. Note that workflow steps can use variable `step_name` to act (slightly) differently for different workflows. For example,

In [12]:
%run mouse
[mouse_20,human_20]
reference = "/path/to/mouse/reference" if step_name.startswith('mouse') else "/path/to/human/reference"

print(reference)

INFO: Running mouse_20: 


/path/to/mouse/reference


Here the variable `step_name` is `mouse_20` or `human_20` depending on the workflow being executed, and expression `mouse_reference if step_name.startswith('mouse') else human_reference` returns `mouse_reference` if the workflow `mouse` is executed, and `human_reference` otherwise.

## Subworkflow

Although workflows are defined separately with all their steps, they do not have to be executed in their entirety. A `subworkflow` refers to a workflow that is defined from one or more steps of an existing workflows. It is specified using syntax `workflow[_steps]` where step can be `n` (step `n`), `:n` (up to `n`), `n:m` (step `n` to `m`) and `m:` (from `m`). For example

  ```
  A              # complete workflow A
  A:5-10         # step 5 to 10 of A
  A:50-          # step 50 up
  A:-10          # up to step 10 of A
  A:10           # step 10 of workflow A can be considered a subworkflow
  ```

## Combined workflow

You can also combine subworkflows to execute multiple workflows one after another. For example,

```python
A + B          # workflow A, followed by B
A_0 + B        # step 0 of A, followed by B
A:-50 + B + C  # up to step 50 of workflow A, followed by B, and C
```

This syntax can be used from the command line (option `workflow`, e.g. `sos-runner myscript.sos align+call`) or used to execute [nested workflows](#nested-workflow) inside the SoS script. 

It is worth noting that combined workflow might work differently from when they are executed individually (e.g. default input of `B` is changed from empty to output of `A_0`), and it is up to the user to resolve conflicts between them.

## Nested workflow

SoS also supports nested workflow in which a complete workflow is treated as part of a step process.
The workflow is execute by SoS action `sos_run`, e.g.

```
sos_run('A')                       # execute workflow A
sos_run('A + B')                   # execute workflow B after A
sos_run('D:-10 + C')               # execute up to step 10 of D and workflow C
sos_run('${aligner} + ${caller}')  # execute user-specified aligner and caller workflows
```

In its simplest form, nested workflow allows you to define another workflow from existing ones. For example,

In [13]:
[align_10]
[align_20]
[call_10]
[call_20]
[default]
sos_run('align+call')

INFO: Running default_0: 


defines a nested workflow that combines workflows `align` and `call` so that the workflow will by default execute two workflows, but can also execute one of them as separate workflows `align` and `call`.

Nested workflow also allows you to define multiple mini-workflows and connect them freely. For example

```python
[a_1]
[a_2]
[b]
[c]
[d_1]
sos_run('a+b')
[d_2]
sos_run('a+c')
```

defines workflows `d` that will execute steps `d_1`, `a_1`, `a_2`, `b_0`, `d_2`,  `a_1`, `a_2`, and `c_0`. 

Nested workflows, like other SoS actions, can be executed repeatedly, for example,

```
[b_1]
[b_2]
[b_3]

[a]
parameters = range(20)
input: 'some.txt', for_each='parameters'
output: '${input}_${_parameters}.res'
sos_run('b')
```

would execute the complete workflow `b` 20 times each with a different parameter. Similarly you can let the nested workflow process groups of input files.

Nested workflows can also be used to compose workflows from user-provided options through command line arguments, configuration files, and even results from previous steps. For example, the following example

```
# aligner steps to use to align the reads 
parameter: aligner = CONFIG.get('aligner', 'bwa')

[bwa_1]
[bwa_2]
[novaalign_1]
[novaalign_2]

[align]
sos_run(aligner)
```

defines workflows `bwa` and `novaalign` to align raw reads. The `align` workflow is a master workflow that executes `bwa` or `novaalign` determined by option `aligner` defined in a configuration file (command line option `-c`) and command line option `--aligner`.